# Global Oceans 2020 hands-on tutorial

## Part 5: Software LBL

In [ ]:
import math
import numpy as np
from scipy.optimize import minimize
from arlpy.plot import figure, scatter, plot
from unetpy import *

### Connect to node 1:

In [ ]:
modem1 = UnetSocket('localhost', 1101)
gw1 = modem1.getGateway()
node1 = modem1.agentForService(Services.NODE_INFO)

In [ ]:
node1.location

### Connect to node 2:

In [ ]:
modem2 = UnetSocket('localhost', 1102)
gw2 = modem2.getGateway()
node2 = modem2.agentForService(Services.NODE_INFO)

In [ ]:
node2.location

### Connect to node 3:

In [ ]:
modem3 = UnetSocket('localhost', 1103)
gw3 = modem3.getGateway()
node3 = modem3.agentForService(Services.NODE_INFO)

In [ ]:
node3.location

### Plot the locations on a map:

In [ ]:
locations = [node1.location[:2], node2.location[:2], node3.location[:2]]

In [ ]:
with figure(xlim=(-25,2025), ylim=(-25,2025), width=800, height=400):
    for loc in locations:
        scatter(loc[0], loc[1], marker='.', size=30)

### Get ranging service on both nodes:

In [ ]:
ranging_node1 = modem1.agentForService(Services.RANGING)
ranging_node2 = modem2.agentForService(Services.RANGING)
ranging_node3 = modem3.agentForService(Services.RANGING)
gw1.subscribe(ranging_node1)
gw2.subscribe(ranging_node2)
gw3.subscribe(ranging_node3)

### Get range from node 1 to node 4:

In [ ]:
# flush any old pending messages
gw1.flush()

In [ ]:
ranging_node1 << RangeReq(to=4)

In [ ]:
rnf1 = gw1.receive(RangeNtf, timeout=5000)
if rnf1 is not None:
    range1 = rnf1.range
    print(range1)
else:
    print('Timed out, try again.')

### Get range from node 2 to node 4:

In [ ]:
# flush any old pending messages
gw2.flush()

In [ ]:
ranging_node2 << RangeReq(to=4)

In [ ]:
rnf2 = gw2.receive(RangeNtf, timeout=5000)
if rnf2 is not None:
    range2 = rnf2.range
    print(range2)
else:
    print('Timed out, try again.')

### Get range from node 3 to node 4:

In [ ]:
# flush any old pending messages
gw3.flush()

In [ ]:
ranging_node3 << RangeReq(to=4)

In [ ]:
rnf3 = gw3.receive(RangeNtf, timeout=5000)
if rnf3 is not None:
    range3 = rnf3.range
    print(range3)
else:
    print('Timed out, try again.')

### Find location of node 4:

$$
\mathbf{x}^* = \arg\min_\mathbf{x} \sum_i \left(\left|\;\mathbf{x}-\mathbf{p}_i\;\right|-r_i\right)^2
$$

In [ ]:
def norm(x):
    return math.sqrt(np.sum(x**2))

def cost(x):
    loc = np.asarray(locations)
    return (norm(x-loc[0])-range1)**2 + (norm(x-loc[1])-range2)**2 + (norm(x-loc[2])-range3)**2

In [ ]:
soln = minimize(cost, (-10,20))
list(soln.x)

### Plot the solution on the map:

In [ ]:
with figure(xlim=(-25,2025), ylim=(-25,2025), width=800, height=400):
    for loc in locations:
        scatter(loc[0], loc[1], marker='.', size=30, hold=True)
        scatter(list(soln.x)[0], list(soln.x)[1], marker='*', size=50, hold=True)

In [ ]:
modem1.close()
modem2.close()
modem3.close()